In [1]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select

from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import traceback
import types

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [4]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba_eyelash = Alibaba(lid, lpwd, browser=browser)
alibaba_eyelash.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [8]:
products = alibaba_eyelash.get_posted_product_info(99)

primary 打开 产品管理 页面 ... ...
primary 切换 至 显示全部产品 ... ...
primary 查找 全部产品列表 第 1 页
primary 完成 查找！
primary 查找 全部产品列表 第 2 页
primary 完成 查找！
primary 查找 全部产品列表 第 3 页
primary 完成 查找！
primary 查找 全部产品列表 第 4 页
primary 完成 查找！
primary 查找 全部产品列表 第 5 页
primary 完成 查找！
primary 查找 全部产品列表 第 6 页
primary 完成 查找！
primary 查找 全部产品列表 第 7 页
primary 完成 查找！
primary 查找 全部产品列表 第 8 页
primary 完成 查找！
primary 查找 全部产品列表 第 9 页
primary 完成 查找！
primary 查找 全部产品列表 第 10 页
primary 完成 查找！
primary 查找 全部产品列表 第 11 页
primary 完成 查找！
primary 查找 全部产品列表 第 12 页
primary 完成 查找！
primary 查找 全部产品列表 第 13 页
primary 完成 查找！
primary 查找 全部产品列表 第 14 页
primary 完成 查找！
primary 查找 全部产品列表 第 15 页
primary 完成 查找！
primary 查找 全部产品列表 第 16 页
primary 完成 查找！
primary 查找 全部产品列表 第 17 页
primary 完成 查找！
primary 查找 全部产品列表 第 18 页
primary 完成 查找！
primary 查找 全部产品列表 第 19 页
primary 完成 查找！
primary 查找 全部产品列表 第 20 页
primary 完成 查找！
primary 查找 全部产品列表 第 21 页
primary 完成 查找！
primary 查找 全部产品列表 第 22 页
primary 完成 查找！
primary 查找 全部产品列表 第 23 页
primary 完成 查找！
primary 查找 全部产品列表 第 24 页
primary 完

In [9]:
len(products)

818

In [14]:
%store products

Stored 'products' (list)


In [2]:
%store -r products
len(products)

818

In [5]:
successed = {}
excepted = {}
running_tasks = {}

In [ ]:
data = {}
data['product_detail'] = {}
data['product_detail']['fp'] = {}
data['product_detail']['lp'] = '<img src="//sc01.alicdn.com/kf/HTB15JbiB2iSBuNkSnhJq6zDcpXaX/231186930/HTB15JbiB2iSBuNkSnhJq6zDcpXaX.jpg" alt="faq and contact" ori-width="750" ori-height="927" />'


In [17]:
# start-process celery -ArgumentList "worker -A libs.CeleryTasks.tasks -c 4 -l info -Q Eyelashes_PM,celery -n Eyelashes[alibaba]@localhost"
# data = {'price': {'isTieredPricing': False, 'price_range':[0.8, 1.1]}}

data = {"ali_id": "60763603063", "detail_pictures": []}
pic = {}
pic["old"] = "//sc02.alicdn.com/kf/HTB1Th0uqVOWBuNjy0Fiq6xFxVXal/231186930/HTB1Th0uqVOWBuNjy0Fiq6xFxVXal.jpg"
pic["new"] = "//sc01.alicdn.com/kf/HTB18_QhB5OYBuNjSsD4q6zSkFXag/231186930/HTB18_QhB5OYBuNjSsD4q6zSkFXag.jpg"
data["detail_pictures"].append(pic)

model = '31c'

count = 0
for p in products:
    ali_id = p['ali_id']
#     if p['pid'].lower == '31a06':
#         continue
    if ali_id in excepted:
        continue
    if ali_id in successed:
        continue
        
    if p['pid'].lower().startswith(model):
        count += 1
        print(count, ':')
        data['ali_id'] = ali_id
        running_tasks[ali_id] = tasks.alibaba_update_product.apply_async(args=[data], queue='Eyelashes_PM')
        
        while True:
            finished = []
            for key in running_tasks:
                task_result = running_tasks[key]
                if task_result.ready():
                    print(task_result.get())
                    if task_result.get()[1]:
                        successed[key] = p
                    finished.append(key)
                    
            for key in finished:
                print(key, running_tasks)
                del running_tasks[key]
            
            if len(running_tasks.keys()) == 3:
                print('wait and try ..., ')
                time.sleep(1)
                continue
            else:
                print('')
                break

1 :
['60763603063', True]
['60795768066', True]
60763603063 {'60763603063': <AsyncResult: 89562a3e-bca9-4514-bef9-033ab93c21aa>, '60795768066': <AsyncResult: 6ec416c4-7115-4170-ace8-74667277731b>, '60762691566': <AsyncResult: b5528401-b409-4ef1-90ab-9b3c5e427891>}
60795768066 {'60795768066': <AsyncResult: 6ec416c4-7115-4170-ace8-74667277731b>, '60762691566': <AsyncResult: b5528401-b409-4ef1-90ab-9b3c5e427891>}

2 :

3 :
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
['60764043674', True]
60764043674 {'60762691566': <AsyncResult: b5528401-b409-4ef1-90ab-9b3c5e427891>, '60762853342': <AsyncResult: 57a7fbd4-2441-46ba-bdc2-303d4e8290c2>, '60764043674': <AsyncResult: 86916910-956f-4

In [13]:
successed

{'60762687569': {'href': '//www.alibaba.com/product-detail/Customizable-Packaging-Boxes-Easy-Application-Magnetic_60765324185.html',
  'title': 'customizable packaging boxes easy application magnetic eyelashes',
  'pid': '31A05',
  'category': 'Magnetic Eyelashes>31A Series',
  'update': '2018-09-02',
  'price': 'US $1.2 - 1.5 / Pair',
  'quality': ['精品'],
  'tags': [],
  'ali_id': '60765324185'},
 '60762845400': {'href': '//www.alibaba.com/product-detail/Wholesale-magnetic-eyelashes-3-magnet-custom_60761755703.html',
  'title': 'wholesale magnetic eyelashes 3 magnet custom handmade dual magnetic eyelashes and 3magnet lashes with no moq',
  'pid': '31A07',
  'category': 'Magnetic Eyelashes>31A Series',
  'update': '2018-09-02',
  'price': 'US $1.2 - 1.5 / Pair',
  'quality': ['精品'],
  'tags': [],
  'ali_id': '60761755703'}}

In [15]:
data = {"ali_id": "60764039619", "detail_pictures": []}
pic = {}
pic["old"] = "//sc02.alicdn.com/kf/HTB1Th0uqVOWBuNjy0Fiq6xFxVXal/231186930/HTB1Th0uqVOWBuNjy0Fiq6xFxVXal.jpg"
pic["new"] = "//sc01.alicdn.com/kf/HTB18_QhB5OYBuNjSsD4q6zSkFXag/231186930/HTB18_QhB5OYBuNjSsD4q6zSkFXag.jpg"
data["detail_pictures"].append(pic)
tasks.alibaba_update_product.apply_async(args=[data], queue='Eyelashes_PM')

<AsyncResult: 3b98e738-4642-4cdb-9af1-c727af39975e>

In [3]:
products[0]

{'href': '//www.alibaba.com/product-detail/OEM-service-custom-eyelash-packaging-false_60769492342.html',
 'title': 'oem service custom eyelash packaging false eyelashes mink fur lashes 3d mink eyelashes',
 'pid': '11A49',
 'category': '3D Mink Eyelashes>11A Series',
 'update': '2018-09-03',
 'price': 'US $1.9 - 2.3 / Pair',
 'quality': ['精品'],
 'tags': [],
 'ali_id': '60769492342'}